# Spotify Exploratory Data Analysis
**Jacob Torres**

In [1]:
"""Imports"""

# Environment variables and authorization
import os
from dotenv import load_dotenv
from spotipy.oauth2 import SpotifyOAuth
from spotipy import Spotify

# Data collection and cleansing
import sqlite3
import numpy as np
import pandas as pd

---
## Data Collection
### SQLite3 Database

In [2]:
# Load Spotify data from local db
DB_FILE = '../app/spotify_db.sqlite3'
DATA_TABLE_QUERY = 'select * from data;'
conn = sqlite3.connect(DB_FILE)

# Dataframe of unliked songs
unliked_df = pd.read_sql(DATA_TABLE_QUERY, conn)
unliked_df = unliked_df.sample(10000)
conn.close()

print(f"Unliked songs: {unliked_df.shape}")
unliked_df.head(10)

Unliked songs: (10000, 14)


,name,id,acousticness,danceability,duration_ms,energy,tempo,instrumentalness,key,liveness,loudness,mode,valence,speechiness
43126,"Etude in F-Sharp Major, Op. 42, No. 4: Andante...",2ONyTm6Wgj8bfhDOGHwYeJ,0.99400,0.377,160107,0.0161,66.910,0.889000,6,0.1150,-27.735,1,0.250,0.0344
159324,Turiya,50XZdKhxzwHBfHEaC6iKtF,0.92600,0.340,257120,0.2970,104.455,0.259000,0,0.1190,-19.953,0,0.454,0.0500
88051,Lenny,1MIDCw2NTW3Dtn9c6ohSC8,0.76200,0.521,297160,0.2000,117.811,0.433000,8,0.1380,-15.759,1,0.348,0.0407
37789,Who Hurt You?,23c9gmiiv7RCu7twft0Mym,0.58600,0.649,231964,0.4930,104.542,0.000167,11,0.0667,-7.645,0,0.277,0.1100
137875,Stay Away from My Friends,3vvBMjyD7ZesKHe1k00gxO,0.27900,0.517,281800,0.8190,77.496,0.000000,9,0.1040,-4.674,1,0.319,0.0280
86970,Argue,3DrzeqjTR3O9zCBdeSRoYu,0.00194,0.557,177667,0.7060,115.122,0.000000,0,0.3200,-7.592,1,0.812,0.0369
44247,Uska Dara - A Turkish Tale,4dV0iVh7Yr0yah9UoetJv0,0.67600,0.758,188867,0.4690,99.340,0.000000,8,0.3450,-8.873,0,0.794,0.0449
131809,Reel By Reel - 2001 Digital Remaster,0SSZBNAnxS3fuhwxJYwgb8,0.03290,0.646,227173,0.8750,137.003,0.000000,10,0.1280,-4.691,1,0.661,0.0327
46759,You Say,3T5P46YHP2llIgppIjyx9M,0.62800,0.500,222693,0.5170,146.194,0.002990,11,0.1130,-10.666,1,0.435,0.0299
45184,Have You Met Miss Jones?,46JqjQRnB54xVejTR7VUUQ,0.62500,0.648,151920,0.2030,119.612,0.000000,5,0.2850,-11.822,0,0.496,0.0327


### SpotifyAPI

In [3]:
# Authorize access to Spotify API
load_dotenv()
cid = os.getenv('CLIENT_ID')
cs = os.getenv('CLIENT_SECRET')
rduri = os.getenv('REDIRECT_URI')
scope = [
    'playlist-read-private',
    'playlist-modify-public',
    'user-library-read',
    'user-read-recently-played'
]

assert cid is not None and cs is not None and rduri is not None, 'One or more environment variables are missing.'

In [4]:
auth_manager = SpotifyOAuth(
    client_id=cid,
    client_secret=cs,
    redirect_uri=rduri,
    scope=scope
)
sp = Spotify(auth_manager=auth_manager)

sp

In [5]:
# Functions for data collection via the API
def get_tracks(index=0):
    """
    Get 50 tracks and track details from current user library starting at given index position.

    returns ids, names, tracks, audio_features -- track details
    """
    tracks = sp.current_user_saved_tracks(limit=50, offset=index)['items']
    names = [track['track']['name'] for track in tracks]
    ids = [track['track']['id'] for track in tracks]
    audio_features = []

    try:
        for id in ids:
            audio_features.append(
                sp.audio_features(id)
            )

        return ids, names, tracks, audio_features

    except:
        return ids, names, tracks, None


def get_new_likes(num=50, index=0):
    """
    Collects features for 50 tracks, starting at given index.

    returns new_likes -- dataframe of num liked tracks and audio features
    """
    ids, names, tracks, audio_features = get_tracks(index=index)

    # Create dataframe of new likes and audio features
    columns = [
        'name',
        'id',
        'duration_ms',
        'danceability',
        'energy',
        'key',
        'loudness',
        'mode',
        'speechiness',
        'acousticness',
        'instrumentalness',
        'liveness',
        'valence',
        'tempo'
    ]

    new_likes = pd.DataFrame(columns=columns)

    # Create list of track dicts with audio features
    if audio_features is None:
        print('Something went wrong.')

    elif len(audio_features) > 0:
        track_data = []
        for i, features in enumerate(audio_features):
            track = {
                'name': names[i],
                'id': features[0]['id'],
                'duration_ms': features[0]['duration_ms'],
                'danceability': features[0]['danceability'],
                'energy': features[0]['energy'],
                'key': features[0]['key'],
                'loudness': features[0]['loudness'],
                'mode': features[0]['mode'],
                'speechiness': features[0]['speechiness'],
                'acousticness': features[0]['acousticness'],
                'instrumentalness': features[0]['instrumentalness'],
                'liveness': features[0]['liveness'],
                'valence': features[0]['valence'],
                'tempo': features[0]['tempo']
            }

            track_data.append(track)

        new_likes = new_likes.append(
            track_data,
            ignore_index=True,
            verify_integrity=True
        )

    return new_likes

In [6]:
ids, names, tracks, audio_features = get_tracks()

assert (
    len(ids) == 50 and
    len(names) == 50 and
    len(tracks) == 50 and
    len(audio_features) != 0
), 'Something went wrong.'

In [7]:
# Get audio features of the tracks
audio_features[0]
#[print(i, features[0]['id']) for i, features in enumerate(audio_features)]

[{'danceability': 0.853,
  'energy': 0.804,
  'key': 10,
  'loudness': -4.25,
  'mode': 0,
  'speechiness': 0.149,
  'acousticness': 0.164,
  'instrumentalness': 0.0157,
  'liveness': 0.214,
  'valence': 0.708,
  'tempo': 89.937,
  'type': 'audio_features',
  'id': '5PSgIaJp6mjIoO5y8kNkrK',
  'uri': 'spotify:track:5PSgIaJp6mjIoO5y8kNkrK',
  'track_href': 'https://api.spotify.com/v1/tracks/5PSgIaJp6mjIoO5y8kNkrK',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/5PSgIaJp6mjIoO5y8kNkrK',
  'duration_ms': 184000,
  'time_signature': 4}]

---
## Data Wrangling and Exploration

In [8]:
%%time
# Add liked tracks to dataframe
indices = np.random.randint(50, 5000, 1000)
liked_df = get_new_likes()

for index in indices:
    new_likes = get_new_likes(index=index)
    liked_df = liked_df.append(new_likes, ignore_index=True, verify_integrity=True)

print(f"Liked songs: {liked_df.shape}")
liked_df.tail(10)

(6509, 14)
Wall time: 12min 28s


,name,id,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
6499,My Name Is Emily,2XxGD6RVh2mDkA0nsSNKCO,142276,0.630,0.237,7,-15.901,1,0.0695,0.83700,0.00000,0.0821,0.856,119.324
6500,Amy aka Spent Gladiator 1,3cjLFY0mLXqBXsnqNhvKH9,147133,0.657,0.834,10,-7.643,0,0.0792,0.08990,0.21600,0.2340,0.832,156.898
6501,Dark in Here,1gpkRjay3yoFotVbdUz0Bh,203360,0.642,0.602,9,-7.665,0,0.0325,0.36300,0.49500,0.0705,0.310,125.126
6502,Your Belgian Things,1trf9MaKJjFIzuPNC8PaZb,229120,0.713,0.261,2,-11.115,1,0.0389,0.78200,0.02520,0.0987,0.305,117.323
6503,Golden Boy,3TDMylLFIdog58gdTjeauu,196053,0.519,0.436,2,-12.344,1,0.0612,0.81600,0.63800,0.3370,0.838,106.515
6504,Get Famous,5HAqb61lv4ZvoxFGSbenPt,199347,0.678,0.720,6,-6.914,1,0.0289,0.05010,0.03370,0.0488,0.877,125.839
6505,Spent Gladiator 2,0oy1kHKfKbUZUAyWVXBAhL,172000,0.897,0.411,9,-11.831,0,0.0909,0.39200,0.01030,0.0926,0.715,133.499
6506,Matthew 25:21,6JPblBgQGfMPzmrQLO6gbm,350227,0.730,0.102,2,-19.086,1,0.0449,0.20200,0.00895,0.1100,0.153,123.105
6507,Werewolf Gimmick,2FgFA3I01P7YqIL3w0a3uc,154640,0.312,0.833,9,-5.040,1,0.1190,0.00324,0.02060,0.0852,0.628,83.995
6508,Psalms 40:2,3AmKiriJoa8HNT4ZCPIAJL,195667,0.570,0.609,5,-9.160,1,0.0490,0.15500,0.01610,0.0900,0.753,159.697


In [9]:
# Create feature matrix and target vector
df = unliked_df.append(liked_df, ignore_index=True, verify_integrity=True)
df = df.drop(['name', 'id'], axis=1)

In [10]:
df.isnull().sum()

acousticness        0
danceability        0
duration_ms         0
energy              0
tempo               0
instrumentalness    0
key                 0
liveness            0
loudness            0
mode                0
valence             0
speechiness         0
dtype: int64

In [11]:
df.sample(10)

,acousticness,danceability,duration_ms,energy,tempo,instrumentalness,key,liveness,loudness,mode,valence,speechiness
11136,0.154000,0.648,240760,0.8010,78.946,0.000000,5,0.3390,-7.240,0,0.807,0.1670
3966,0.000053,0.355,186787,0.9940,167.972,0.001870,7,0.3540,-2.947,1,0.190,0.1310
9838,0.934000,0.527,230280,0.0624,80.659,0.000010,9,0.1220,-18.425,1,0.132,0.0720
9457,0.960000,0.636,227173,0.0451,132.538,0.053900,8,0.0858,-23.017,1,0.227,0.0625
743,0.567000,0.547,153787,0.1610,133.897,0.658000,8,0.0597,-25.859,1,0.130,0.0310
8701,0.579000,0.503,207030,0.7410,88.335,0.000001,4,0.0712,-5.128,1,0.458,0.0470
6147,0.529000,0.437,352827,0.6510,79.583,0.000022,0,0.9050,-12.979,1,0.801,0.0563
11689,0.058200,0.477,213707,0.4530,149.245,0.675000,4,0.1130,-10.592,0,0.330,0.0471
7884,0.995000,0.382,122827,0.0274,74.081,0.001100,7,0.1460,-24.980,1,0.356,0.0516
10401,0.242000,0.635,163127,0.6590,96.855,0.028500,7,0.1050,-7.499,1,0.340,0.1160


In [12]:
df.describe()

,acousticness,danceability,energy,tempo,instrumentalness,liveness,loudness,valence,speechiness
count,16509.000000,16509.000000,16509.000000,16509.000000,16509.000000,16509.000000,16509.000000,16509.000000,16509.000000
mean,0.473492,0.568886,0.489705,116.647115,0.130290,0.190477,-10.660336,0.519505,0.092229
std,0.354438,0.169857,0.246899,29.978107,0.274141,0.156095,5.171688,0.254610,0.140900
min,0.000001,0.000000,0.000020,0.000000,0.000000,0.019300,-40.153000,0.000000,0.000000
25%,0.114000,0.456000,0.290000,93.867000,0.000000,0.098100,-13.551000,0.317000,0.034300
50%,0.462000,0.584000,0.483000,114.046000,0.000158,0.126000,-9.745000,0.518000,0.044900
75%,0.829000,0.695000,0.684000,134.971000,0.048500,0.233000,-6.795000,0.728000,0.077900
max,0.996000,0.979000,1.000000,218.363000,0.999000,0.998000,3.855000,0.991000,0.967000


---
## Commit Datasets to SQLite3 Database

In [13]:
# Create "liked_songs_jt" table
LIKED_TABLE_QUERY = """create table if not exists liked_songs_jt (
    name varchar(3000),
    id varchar(50),
    duration_ms int,
    danceability float,
    energy float,
    key int,
    loudness float,
    mode int,
    speechiness float,
    acousticness float,
    instrumentalness float,
    liveness float,
    valence float,
    tempo float
)"""

conn = sqlite3.connect(DB_FILE)
cur = conn.cursor()

try:
    cur.execute(LIKED_TABLE_QUERY)
    conn.commit()

except (Exception, sqlite3.Error) as err:
    print(err)

In [14]:
# Insert liked song data into liked_songs_jt table
try:
    liked_df.to_sql('liked_songs_jt', conn, if_exists='replace')

except (Exception, sqlite3.Error) as err:
    print(err)

In [15]:
# Create "train" table in database
TRAIN_TABLE_QUERY = """create table if not exists train (
    acousticness float,
    danceability float,
    duration_ms int,
    energy float,
    tempo float,
    instrumentalness float,
    key int,
    liveness float,
    loudness float,
    mode int,
    valence float,
    speechiness float
)"""

try:
    cur.execute(TRAIN_TABLE_QUERY)
    conn.commit()

except (Exception, sqlite3.Error) as err:
    print(err)

In [16]:
# Insert entire training dataset into train table
try:
    df.to_sql('train', conn, if_exists='replace')

except (Exception, sqlite3.Error) as err:
    print(err)

finally:
    cur.close()
    conn.close()